In [3]:
import os

os.environ["OPENAI_API_KEY"] = "Try_your_Own_api_Key"


In [33]:
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.6)

intro = llm("Give a proper introdection about LangChain under 100 words assuming that I'm a complete beginner ")
print(intro)



LangChain is a decentralized platform that aims to revolutionize the language learning industry by connecting language learners with native speakers through blockchain technology. It offers a secure and efficient way for users to exchange language skills, eliminating the need for intermediaries and providing a more personalized and cost-effective learning experience. With its user-friendly interface and innovative approach, LangChain is poised to disrupt the traditional language learning methods and create a global community of language enthusiasts. Whether you are a beginner or an advanced learner, LangChain offers a unique and immersive way to learn a new language. 


In [12]:
from langchain.prompts import PromptTemplate

Name = PromptTemplate(
    input_variable = ["Place"],
    template = "Name 5 best restaurants from {Place}. I need just the names, no description needed",)

Name.format(Place = "Hyderabad")


'Name 5 best restaurants from Hyderabad. I need just the names, no description needed'

In [30]:
from langchain.chains import LLMChain

chain = LLMChain( llm = llm, prompt = Name)
chain.run("Kerala")

'\n\n1. Paragon Restaurant\n2. Malabar Junction\n3. The Rice Boat\n4. Dhe Puttu\n5. Dal Roti '

## Simple Sequential Chains

In [41]:
llm = OpenAI(temperature = 0.9)

#First Prompt Template for Restaurant Name 
rest_name = PromptTemplate(
    input_variable = ["city"],
    template = "Suggest one best place in {city} for my meal, Just the name and no description"
    )
#First chain for Restaurant Name 
name_chain = LLMChain(llm = llm, prompt = rest_name)

#Second Prompt Template for Restaurant Items 
rest_items = PromptTemplate(
    input_variable = ["items"],
    template = "Any 5 best {items} I can have at this place. Just the name and no description. Return as a Ordered list"
    )

#Second Chain for Restaurant Items 
items_chain = LLMChain(llm = llm, prompt = rest_items)

#Applying Simple Sequential Chains for these two templates

from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, items_chain])
response = chain.run("Irish")
print(response)





1. Dingle Whiskey
2. Dingle Gin
3. Dingle Peninsula
4. Dingle Bay
5. Dingle Aquarium


## Sequential Chain

In [56]:
llm = OpenAI(temperature = 0.9)

#First Prompt Template for Restaurant Name 
rest_name = PromptTemplate(
    input_variable = ["City"],
    template = "Suggest one best restaurant in {City} for my meal, Just the name and no description"
    )
#First chain for Restaurant Name 
name_chain = LLMChain(llm = llm, prompt = rest_name, output_key = "restaurant_name")

#Second Prompt Template for Restaurant Items 
rest_items = PromptTemplate(
    input_variable = ["restaurant_name"],
    template = "Any 5 best items I can have at {restaurant_name}. Just the item names and no description. Return as a Ordered list"
    )

#Second Chain for Menu Items 
items_chain = LLMChain(llm = llm, prompt = rest_items, output_key = "menu_items")

#Applying Sequential Chains for these two templates

from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, items_chain],
    input_variables = ["City"],
    output_variables = ["restaurant_name", "menu_items"])

chain({"City" : "Rajahmundry"})



{'City': 'Rajahmundry',
 'restaurant_name': '\n\nAnnapurna Punjabi Dhaba \n',
 'menu_items': '\n\n1. Butter Chicken\n2. Tandoori Chicken\n3. Saag Paneer\n4. Chole Bhature\n5. Aloo Paratha'}

In [67]:
llm = OpenAI(temperature = 0.9)

#Primary Prompt Template for City Name 
city_name = PromptTemplate(
    input_variable = ["City"],
    template = "Give a very nice short introduction about the best hospital facilities in the {City} under 250 words"
    )
#First chain for City Name 
city_chain = LLMChain(llm = llm, prompt = city_name, output_key = "city_name")

#First Prompt Template for Hospital Name 
hosp_name = PromptTemplate(
    input_variable = ["city_name"],
    template = "Suggest two best hospitals in {city_name} for all sorts of emergencies."
    )
#First chain for Hospital Name 
hospital_chain = LLMChain(llm = llm, prompt = hosp_name, output_key = "hospital_name")  # Fixed variable name from rest_name to hosp_name

#Second Prompt Template for Restaurant Items 
hosp_details = PromptTemplate(
    input_variable = ["hospital_name"],
    template = "Give the introduction about the {hospital_name} and list top 3 best doctors of that {hospital_name}, their degrees and their specialisation. Return as a Ordered list"
    )

#Second Chain for Menu Items 
details_chain = LLMChain(llm = llm, prompt = hosp_details, output_key = "hospital_details")  # Fixed variable name from rest_items to hosp_details

#Applying Sequential Chains for these two templates

from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [city_chain, hospital_chain, details_chain],  # Added closing bracket here
    input_variables = ["City"],
    output_variables = ["city_name","hospital_name", "hospital_details"]
)

chain({"City" : "Visakhapatnam"})

{'City': 'Visakhapatnam',
 'city_name': '\n\nVisakhapatnam, also known as Vizag, is a city on the east coast of India in the state of Andhra Pradesh. It is a major industrial center, but it also boasts some of the best hospital facilities in the region. These facilities are equipped with advanced technology, highly skilled doctors, and compassionate staff, making them the go-to hospitals for medical treatment.\n\nOne of the top hospitals in Visakhapatnam is Apollo Hospitals, which offers a wide range of medical services in various specialties such as cardiology, neurology, oncology, and more. It is known for its state-of-the-art infrastructure, cutting-edge technology, and experienced medical professionals.\n\nAnother renowned hospital in Vizag is Manipal Hospitals, known for its quality healthcare services and patient-centric approach. It has a team of highly qualified doctors and modern facilities such as advanced operation theaters, intensive care units, and diagnostic services.\n\n